In [32]:
import yaml
from sqlalchemy import create_engine
from sqlalchemy import inspect
import data_cleaning as dc
import data_extraction as de
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from data_cleaning import DataCleaning
import pandas as pd

dab = DatabaseConnector('db_creds.yaml')
dab.init_db_engine()
#List the tables
tables = dab.list_db_tables()
#prints tables
print(tables)




['legacy_store_details', 'legacy_users', 'orders_table']


In [33]:
de = DataExtractor()
original_df = de.extract_user_data()

#Copy the data from the table into a Pandas DF
#pandaDF = dab.read_db_creds()
#for col in pandaDF.columns:
    #print(col)

#db_yaml_file='db_creds.yaml'
#db_connector=DatabaseConnector(db_yaml_file)

pd.set_option('display.max_columns', None)
legacy_users = de.read_rds_table('legacy_users')
legacy_users.head(5)


,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


In [34]:
legacy_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          15320 non-null  int64 
 1   first_name     15320 non-null  object
 2   last_name      15320 non-null  object
 3   date_of_birth  15320 non-null  object
 4   company        15320 non-null  object
 5   email_address  15320 non-null  object
 6   address        15320 non-null  object
 7   country        15320 non-null  object
 8   country_code   15320 non-null  object
 9   phone_number   15320 non-null  object
 10  join_date      15320 non-null  object
 11  user_uuid      15320 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.4+ MB


In [35]:
legacy_users['country'].value_counts()

United Kingdom    9371
Germany           4708
United States     1205
NULL                21
GMRBOMI0O1           1
7ZNO5EBALT           1
3518UD5CE8           1
RQRB7RMTAD           1
PNRMPSYR1J           1
5EFAFD0JLI           1
YOTSVPRBQ7           1
50KUU3PQUF           1
EWE3U0DZIV           1
XN9NGL5C0B           1
S0E37H52ON           1
XGI7FM0VBJ           1
AJ1ENKS3QL           1
I7G4DMDZOZ           1
T4WBZSW0XI           1
Name: country, dtype: int64

In [36]:
# There are many that are not actually a country. There are only three correct values. 

a = ['United Kingdom','United States','Germany']
cleaned_users = legacy_users[legacy_users.country.isin(a)]
cleaned_users['country'].value_counts()

United Kingdom    9371
Germany           4708
United States     1205
Name: country, dtype: int64

In [37]:
cleaned_users['country_code'].value_counts()

GB     9365
DE     4708
US     1205
GGB       6
Name: country_code, dtype: int64

In [38]:
cleaned_users['country_code'].replace(to_replace='GGB',value='GB', inplace=True)
cleaned_users['country_code'].value_counts()

C:\Users\yurir\AppData\Local\Temp\ipykernel_28012\1802846815.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_users['country_code'].replace(to_replace='GGB',value='GB', inplace=True)


GB    9371
DE    4708
US    1205
Name: country_code, dtype: int64

In [43]:
dobs = print(cleaned_users['date_of_birth'])


0        1990-09-30
1        1940-12-01
2        1995-08-02
3        1972-09-23
4        1952-12-20
            ...    
15315    1943-08-09
15316    1948-08-20
15317    1940-10-09
15318    1952-06-04
15319    1994-03-27
Name: date_of_birth, Length: 15284, dtype: object


In [44]:
#Very large data set with, lets see if value count will work? 

dobs = print(cleaned_users['date_of_birth'].value_counts())

1958-02-02    6
1978-01-31    5
1956-07-16    5
1961-03-27    5
1951-01-09    5
             ..
1938-12-22    1
2005-03-28    1
1963-04-26    1
1964-07-13    1
1994-03-27    1
Name: date_of_birth, Length: 11344, dtype: int64


In [49]:
#need another way to represent data
#will count what how many values have varring sets of dob

dobs=cleaned_users['date_of_birth']
dobs.map(lambda calc: len(calc)).value_counts()


10    15268
15        6
16        5
12        4
11        1
Name: date_of_birth, dtype: int64

In [51]:
cleaned_users = cleaned_users[dobs.apply(lambda x: len(x)==10)]
dobs=cleaned_users['date_of_birth']
dobs.map(lambda calc: len(calc)).value_counts()

C:\Users\yurir\AppData\Local\Temp\ipykernel_28012\1233488452.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cleaned_users = cleaned_users[dobs.apply(lambda x: len(x)==10)]


10    15268
Name: date_of_birth, dtype: int64

In [55]:
phone_series = cleaned_users['phone_number']
phone_series2 = phone_series.apply(lambda x: ''.join([i for i in x if str.isnumeric(i)])[-10:])
cleaned_users['phone_number']=phone_series2
print(phone_series2)

0        0047905356
1        1614960674
2        1214960340
3        3069990871
4        1214960225
            ...    
15315    0292018946
15316    1144960977
15317    0298408192
15318    2397113836
15319    1314960870
Name: phone_number, Length: 15268, dtype: object


In [ ]:
import tabula
import pandas as pd
 
def retrieve_pdf_data(url: str):
    '''takes PDF url as input and outputs PDF data as table'''
    dfs =tabula.read_pdf(url,pages='all')
    card_data=pd.concat(dfs,ignore_index=True)
    print('number of dataframes created:')
    print(len(dfs))
    return card_data

retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')